# Are the recalls more schematic than the stories themselves?

## Based on 33



## Loc_corr_subj_i_story_j - Loc_corr_story_j

#### finding each participants avg loc/soc correlation and then subtracting the loc_corr_story

#### 4/8/21


In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as py
import plotly
import os
import glob
import math
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import copy
from random import randrange
from sklearn.metrics import jaccard_score
import random
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression
from scipy import stats


from sklearn.decomposition import PCA #for cluster analysis
from gensim.models import KeyedVectors #for word embeddings
import matplotlib.pyplot as plt #for plotting
import os #for importing
import pickle #for loading transcripts
from scipy.stats import pearsonr 

# from _DRAFT_20200604_functions import * #includes constants and score function
from tqdm import tqdm_notebook #for progress bar

#Import wordcloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

%autosave 5

Autosaving every 5 seconds


# 0. Import recalls, and uncentered story and template vectors and sums 

In [3]:
# import recalls and sums
recalls = pickle.load( open( 'fr_recalls', "rb" ) )
sums = pickle.load( open( "fr_sums", "rb" ) )

#import individual words of stories
granstories = pickle.load( open( 'gran_stories', "rb" ) )

#import non-centered story, template
templates = pickle.load( open( 'template_vectors', "rb" ) )
stories = pickle.load( open( 'actual_story_vectors', "rb" ) )

## Word to Vec

In [4]:
wikipath = 'rolando/wiki-news-300d-1M.vec'
wv_model = KeyedVectors.load_word2vec_format(wikipath)

In [5]:
wv_dim = 300

# FastText preprocessing, based on bittlingmayer/ft_wiki_preproc.py
# Remove special characters, put spaces between all tokens
SUB = ["s/’/'/g", "s/′/'/g", "s/''/ /g", "s/'/ ' /g", 's/“/"/g', 's/”/"/g', 's/"/ /g', "s/\\./ \\. /g", "s/<br \\/>/ /g", "s/, / , /g", "s/(/ ( /g", "s/)/ ) /g", "s/\\!/ \\! /g", "s/\\?/ \\? /g", "s/\\;/ /g", "s/\\:/ /g", "s/-/ - /g", "s/=/ /g", "s/=/ /g", "s/*/ /g", "s/|/ /g", "s/«/ /g", 
       "s/…/ /g", "s/‘/ /g", "s/í/ /g", "s/ñ/ /g", "s/\x84/ /g", "s/î/ /g", "s/ó/ /g", "s/\x83/ /g", "s/ï/ /g", "s/õ/ /g",
       "s/ò/ /g", "s/,/ /g", "s/ô/ /g", "s/\x92/ /g", "s/é/ /g", "s/\x8e/ /g", "s/â\x80¦/ /g", "s/\x91/ /g", "s/\x93/ /g",
       "s/\x94/ /g", "s/ã®/ /g", "s/ã¨/ /g", "s/ã©/ /g",
       "s/\â\x80\x99/ /g", "s/â\x80\x9c/ /g", "s/â\x80\x9d/ /g", "s/â\x80\x99/ /g", "s/â\x80\x9c/ /g", "s/â\x80\x98/ /g",
       "s/â/ /g"]

def __normalize_text(s):
    for sg in SUB:
        rep = sg.replace('\\','').split('/')
        s = s.replace(rep[1], rep[2])
    s = s.replace('/',' ')
    return s

def __spaces(s):
    return ' '.join(s.split())

def __digits(s):
    return ''.join(filter(lambda c: not c.isdigit(), s))

# def preproc(s):
#     return __punctuation(__spaces(__digits(__normalize_text(s.lower()))))

def preproc(s):
    return (__spaces(__digits(__normalize_text(s.lower()))))

def __punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))

def word2vecSent(sentence, model = 'fasttext'):
    wv_dim = 300 #for glove and fasttext
    
    if model == 'glove':
        wvmodel = glove_model
    elif model == 'fasttext':
        wvmodel = wv_model
        
    words = preproc(sentence).split(' ')
    wv = np.zeros((len(words), wv_dim))
    for i in range(len(words)):
        if words[i] in wvmodel.vocab:
            wv[i,:] = wvmodel.word_vec(words[i])
    
    return words, wv


# Recalls: Concatenate words and word vectors in parallel across all participants in a story

In [79]:
# 0 for np
# 1 for loc
# 2 for soc

### need to change naming of .csv files as well at the bottom

this_priming_group = 0

In [80]:
all_words = {}
all_wvs = {}
#priming type order
all_priming = {}

for key in recalls.keys():
    all_words[key] = {}
    all_wvs[key] = {}

#iterate through all np recalls in a story and concatente words for each participant
## keeping each participant distinct
for key in recalls:
    for j in range(0,len(recalls[key][this_priming_group])):
        p_words, p_wvs = word2vecSent(recalls[key][this_priming_group][j][0])
        # reshape p_words
        p_words = np.array(p_words)
        p_words = p_words.reshape(p_words.shape[0],-1)
        all_words[key][j] = p_words
        all_wvs[key][j] = p_wvs

# 2. Centering all words + story wvs + template wvs

In [81]:
#Concatenate all words from recall

concat = np.zeros((0,300))

for story in all_wvs:
    for key in all_wvs[story]:
        for i in range(0, len(all_wvs[story][key])):
            concat = np.vstack((concat, all_wvs[story][key][i]))
            
print(concat.shape)
    
for key in granstories:
    concat = np.vstack((concat, granstories[key]['wvs']))
    
for key in templates:
    concat = np.vstack((concat, templates[key]))

centering_vec = np.mean(concat, axis = 0)

# template vectors

for key in templates:
    templates[key] = templates[key] - centering_vec
    
# recall vectors

for key in recalls:
    for i in range(0, len(all_wvs[key])):
        all_wvs[key][i] = all_wvs[key][i] - centering_vec
            
#story vectors

#make new dict with int key instead of string
for key in granstories:
    granstories[key]['wvs'] = granstories[key]['wvs'] - centering_vec

(12190, 300)


## template regression

In [82]:
schema_order = [1, 2, 3, 4, 10, 20, 30, 40]

new_templates = {}

for template in templates:
    new_input = np.zeros((300,0))
    for other_template in schema_order:
        if other_template == template:
            pass
        else:
            new_input = np.concatenate((new_input, templates[other_template].reshape(templates[other_template].shape[0],-1)), axis = 1)
    ## regression 
    output = np.copy(templates[template])
    #output = output.reshape(output.shape[0],-1)
    model = LinearRegression().fit(new_input, output)
    ## Is subtracting the intercept correct?? 
    new_vec = output - np.dot(model.coef_, new_input.T)- model.intercept_
    new_templates[template] = new_vec

# *LOC* correlations with recalls
## 1. subtract loc_stories[ley] from avg loc cor with recall words

In [83]:
new_wvs = copy.deepcopy(all_wvs)

In [91]:
### A. Correlate each word with the LOC WV
loc_cors = {}
#loc_parts = np.array(['dif'], dtype=object)
loc_parts = np.zeros((0,1))

for key in all_wvs:
    loc = key%10
    this_loc = new_templates[loc].reshape(new_templates[loc].shape[0], -1)
    for partc in all_wvs[key]:
        n_words = all_wvs[key][partc].shape[0]
        this_loc_cors = np.zeros((n_words,1))
        for i in range(0, n_words):
            this_wv = new_wvs[key][partc][i].reshape(new_wvs[key][partc][i].shape[0], -1)
            this_correl = np.corrcoef(this_loc.T, this_wv.T)
            this_loc_cors[i] = this_correl[0,1]
        print(this_loc_cors)
        mean_loc_cors = np.mean(this_loc_cors)
        loc_parts = np.vstack((loc_parts, mean_loc_cors))

[[ 0.02459234]
 [ 0.05099237]
 [ 0.06595787]
 [-0.10110041]
 [ 0.06764551]
 [ 0.01170985]
 [-0.07001709]
 [ 0.55031354]
 [-0.01549156]
 [ 0.05099237]
 [-0.02407475]
 [-0.07264855]
 [-0.07001709]
 [ 0.40968506]
 [-0.06494861]
 [-0.05046712]
 [-0.03198913]
 [ 0.05218882]
 [-0.0409034 ]
 [-0.0033286 ]
 [ 0.15574601]
 [-0.05046712]
 [ 0.37203507]
 [-0.10110041]
 [-0.07001709]
 [ 0.19141572]
 [-0.13283703]
 [-0.05046712]
 [-0.1590224 ]
 [-0.0661768 ]
 [-0.05046712]
 [ 0.34975788]
 [-0.0409034 ]
 [-0.02321117]
 [-0.0033286 ]
 [ 0.08187424]
 [-0.07264855]
 [ 0.06764551]
 [-0.01549156]
 [-0.0033286 ]
 [ 0.1100683 ]
 [-0.07264855]
 [-0.09811463]
 [-0.15621258]
 [-0.05710184]
 [-0.02914774]
 [-0.0409034 ]
 [-0.0033286 ]
 [ 0.1100683 ]
 [-0.09137452]
 [-0.04905168]
 [-0.07264855]
 [-0.06439519]
 [-0.06965112]
 [ 0.04411405]
 [-0.0409034 ]
 [ 0.01862836]
 [-0.06960266]
 [-0.05046712]
 [ 0.37203507]
 [ 0.15574601]
 [ 0.00863242]
 [-0.01161228]
 [ 0.0066028 ]
 [ 0.01944116]
 [-0.03283685]
 [ 0.02632

[[-0.04740986]
 [-0.0429163 ]
 [-0.05785589]
 [-0.07340729]
 [ 0.02397343]
 [ 0.08684308]
 [ 0.05627347]
 [-0.11667034]
 [ 0.05739935]
 [-0.03752289]
 [-0.04740986]
 [-0.12135918]
 [-0.08722378]
 [-0.08416494]
 [-0.07333472]
 [-0.03752942]
 [-0.06844252]
 [ 0.05547759]
 [ 0.00370995]
 [ 0.00086494]
 [ 0.0457381 ]
 [-0.08277562]
 [-0.07898664]
 [-0.07533623]
 [-0.05025267]
 [-0.04160389]
 [-0.07987418]
 [ 0.03475213]
 [ 0.01360919]
 [ 0.00905797]
 [-0.03752289]
 [-0.05785589]
 [-0.12135918]
 [-0.02706545]
 [-0.07576122]
 [ 0.01759361]
 [-0.0429163 ]
 [-0.04539955]
 [-0.09691294]
 [-0.04160389]
 [-0.09794756]
 [-0.08826094]
 [ 0.03058764]
 [-0.03752942]
 [ 0.10508496]
 [-0.03752289]
 [ 0.06191582]
 [ 0.09357151]
 [ 0.13279605]
 [-0.11667034]
 [ 0.25017237]
 [ 0.02456383]
 [ 0.01776555]
 [-0.0429163 ]
 [-0.00053965]
 [ 0.03475213]
 [ 0.09357151]
 [ 0.0920347 ]
 [-0.01145581]
 [ 0.06832701]
 [ 0.01191456]
 [-0.03752942]
 [ 0.25017237]
 [ 0.21284687]
 [-0.07919801]
 [-0.01801586]
 [ 0.06118

[[-0.11053746]
 [ 0.00346875]
 [ 0.04770199]
 [ 0.07673496]
 [-0.11053746]
 [ 0.54042502]
 [-0.01467965]
 [-0.0157832 ]
 [-0.098517  ]
 [-0.08206919]
 [-0.0108523 ]
 [ 0.04229834]
 [-0.0108523 ]
 [-0.06433116]
 [ 0.02935074]
 [ 0.06424177]
 [-0.0475198 ]
 [-0.04296366]
 [-0.0157832 ]
 [-0.04327322]
 [-0.07652288]
 [ 0.03615359]
 [-0.05719236]
 [ 0.07553282]
 [ 0.03253871]
 [-0.01467965]
 [-0.04965964]
 [-0.1189351 ]
 [ 0.0059747 ]
 [-0.04396293]
 [ 0.00582629]
 [-0.05386288]
 [ 0.01438609]
 [-0.07291217]
 [-0.02025861]
 [-0.02285733]
 [-0.0417548 ]
 [-0.04512174]
 [-0.04273342]
 [ 0.07553282]
 [ 0.03253871]
 [-0.04296366]
 [ 0.03604924]
 [-0.02014999]
 [ 0.0059747 ]
 [ 0.01969313]
 [ 0.16143012]
 [ 0.06424177]
 [ 0.08370321]
 [-0.01467965]
 [ 0.06424177]
 [-0.04095897]
 [-0.05780334]
 [-0.0157832 ]
 [-0.14437915]
 [-0.06749056]
 [-0.05719236]
 [-0.00573427]
 [-0.04296366]
 [ 0.03604924]
 [-0.02014999]
 [ 0.06424177]
 [ 0.11763424]
 [-0.02983514]
 [-0.00609275]
 [-0.02572667]
 [ 0.00597

[[ 0.03523104]
 [ 0.02362188]
 [-0.14106883]
 [-0.05046712]
 [ 0.10431651]
 [ 0.07916812]
 [-0.0661768 ]
 [-0.05046712]
 [ 0.16172666]
 [ 0.02529164]
 [ 0.01715663]
 [-0.13283703]
 [-0.04253199]
 [ 0.06328511]
 [ 0.20818225]
 [-0.13759618]
 [-0.05046712]
 [ 0.19841511]
 [-0.05006556]
 [-0.07264855]
 [-0.05046712]
 [ 0.07048703]
 [-0.0409034 ]
 [-0.0033286 ]
 [ 0.21161376]
 [-0.01161228]
 [-0.02914774]
 [-0.0514292 ]
 [ 0.06328511]
 [ 0.07931351]
 [ 0.0066028 ]
 [-0.06799803]
 [-0.07264855]
 [ 0.07101367]
 [-0.0409034 ]
 [ 0.02529164]
 [ 0.15574601]
 [-0.01161228]
 [-0.0033286 ]
 [ 0.02432376]
 [ 0.00491605]
 [-0.00789059]
 [-0.04960777]
 [-0.07001709]
 [ 0.12447364]
 [-0.0033286 ]
 [ 0.1415245 ]
 [ 0.02529164]
 [ 0.01558351]
 [-0.07001709]
 [-0.05427627]
 [-0.04595008]
 [-0.01206073]
 [ 0.05099237]
 [ 0.08060424]
 [ 0.02529164]
 [ 0.13370498]
 [-0.05046712]
 [ 0.06471737]
 [-0.0661768 ]
 [-0.05046712]
 [ 0.16172666]
 [ 0.12447364]
 [-0.04253199]
 [-0.01549156]
 [-0.01088312]
 [-0.02914

In [87]:
np.mean(loc_parts)

-0.002022446941969812

In [85]:
loc_parts

array([[ 1.48397130e-02],
       [ 1.30475756e-03],
       [ 6.30018203e-03],
       [ 6.54549001e-03],
       [-3.85077088e-03],
       [ 8.82514829e-03],
       [ 2.90004574e-02],
       [ 4.17272817e-03],
       [ 4.46522025e-02],
       [ 1.44213661e-02],
       [-1.57532707e-02],
       [ 8.22857325e-03],
       [-2.39542096e-02],
       [ 3.28714817e-03],
       [-7.42318269e-03],
       [-2.43377690e-02],
       [ 8.70657462e-03],
       [-9.60627737e-04],
       [ 1.08452558e-02],
       [-1.21117753e-03],
       [ 3.87119806e-03],
       [-2.21211753e-03],
       [-3.25221241e-03],
       [-7.58866518e-04],
       [ 1.25509756e-03],
       [-1.79964200e-06],
       [ 3.65449914e-03],
       [-4.09118249e-03],
       [-7.57889712e-03],
       [-4.83536450e-02],
       [-7.01288469e-03],
       [-4.67941975e-03],
       [-1.69532878e-02],
       [ 2.89532570e-03],
       [-3.47990785e-03],
       [ 2.13573846e-04],
       [ 1.19282727e-02],
       [ 2.26760433e-02],
       [ 2.5

# *SOC* correlations with recalls


In [88]:
soc_cors = {}
soc_parts = np.zeros((0,1))

for key in all_wvs:
    soc = round(key/10)*10
    this_soc = new_templates[soc].reshape(new_templates[soc].shape[0], -1)
    for partc in all_wvs[key]:
        n_words = all_wvs[key][partc].shape[0]
        this_soc_cors = np.zeros((n_words,1))
        for i in range(0, n_words):
            this_wv = new_wvs[key][partc][i].reshape(new_wvs[key][partc][i].shape[0], -1)
            this_correl = np.corrcoef(this_soc.T, this_wv.T)
            this_soc_cors[i] = this_correl[0,1]
        mean_soc_cors = np.mean(this_soc_cors)
        soc_parts = np.vstack((soc_parts, mean_soc_cors))

In [90]:
np.mean(soc_parts)

-0.0008340990871695485

In [64]:
for i in range(0, loc_parts.shape[0]):
    this_array = np.array(['soc', 'loc_cor', loc_parts[i][0], pcount])
    all_subs = np.vstack((all_subs, this_array))
    this_array = np.array(['soc', 'soc_cor', soc_parts[i][0], pcount])
    all_subs = np.vstack((all_subs, this_array))
    pcount += 1

In [65]:
all_subs.shape

(630, 4)

In [67]:
all_subs

array([['np', 'loc_cor', '0.011219944942810863', '0'],
       ['np', 'soc_cor', '-0.023949563643471134', '0'],
       ['np', 'loc_cor', '-0.00046457995391973835', '1'],
       ...,
       ['soc', 'soc_cor', '0.008187597809480437', '313'],
       ['soc', 'loc_cor', '0.0033959705723786873', '314'],
       ['soc', 'soc_cor', '0.039734150054403596', '314']], dtype='<U32')

In [66]:
# convert array into dataframe
loc_parts = pd.DataFrame(loc_parts)
soc_parts = pd.DataFrame(soc_parts)

loc_parts.to_csv("loc_cor2.csv",  index = False)
soc_parts.to_csv("soc_cor2.csv",  index = False)

In [68]:
np.savetxt('RECALL_ALL_CORREL.csv', all_subs, delimiter=',', fmt='%s', header='priming,cor_type,cor, sub', comments='')


In [ ]:
all_subs = np.zeros((0,4))
pcount = 0